In [1]:
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import Transformer, RNN, RETAIN, MLP
from pyhealth.tasks import mortality_prediction_mimic3_fn, readmission_prediction_mimic3_fn, drug_recommendation_mimic3_fn, length_of_stay_prediction_mimic3_fn
from pyhealth.trainer import Trainer

dataset = MIMIC3Dataset(
    root='/data/physionet.org/files/mimiciii/1.4/',
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    code_mapping={
        # "ICD9CM": "CCSCM", 
        # "ICD9PROC": "CCSPROC",
        "NDC": ("ATC", {"target_kwargs": {"level": 3}})
        },
    dev=False,
    refresh_cache=False
)


/home/pj20/miniconda3/envs/txgnn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mimic3_ds = dataset.set_task(readmission_prediction_mimic3_fn)

train_dataset, val_dataset, test_dataset = split_by_patient(
    mimic3_ds, [0.8, 0.1, 0.1]
)
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

Generating samples for readmission_prediction_mimic3_fn: 100%|██████████| 46520/46520 [00:00<00:00, 108702.43it/s]


In [34]:
model_w_pre = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    pretrained_emb="LM/sapbert",
    embedding_dim=256,
)

Loading pretrained embedding for conditions...
Loading pretrained embedding for procedures...
Loading pretrained embedding for drugs...


In [35]:
# STEP 4: define trainer
trainer = Trainer(model=model_w_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=15,
    optimizer_params = {"lr": 1e-4},
    monitor="pr_auc",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(4031, 768, padding_idx=0)
    (procedures): Embedding(1276, 768, padding_idx=0)
    (drugs): Embedding(194, 768, padding_idx=0)
  )
  (linear_layers): ModuleDict(
    (conditions): Linear(in_features=768, out_features=256, bias=True)
    (procedures): Linear(in_features=768, out_features=256, bias=True)
    (drugs): Linear(in_features=768, out_features=256, bias=True)
  )
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=256, out_features=256, bias=False)
              (1): Linear(in_features=256, out_features=256, bias=False)
              (2): Linear(in_features=256, out_features=256, bias=False)
            )
            (output_linear): Linear(in_features=256, out_features=256, bias=False)
            (attention): 

Epoch 0 / 15: 100%|██████████| 243/243 [00:03<00:00, 62.86it/s]

--- Train epoch-0, step-243 ---
loss: 1.8513



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 261.69it/s]

--- Eval epoch-0, step-243 ---
pr_auc: 0.6164
roc_auc: 0.6185
f1: 0.6404
loss: 0.7276
New best pr_auc score (0.6164) at epoch-0, step-243


...

--- Train epoch-14, step-3645 ---
loss: 0.6534



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 234.73it/s]

--- Eval epoch-14, step-3645 ---
pr_auc: 0.6619
roc_auc: 0.6526
f1: 0.3598
loss: 0.7386
Loaded best model



Evaluation: 100%|██████████| 32/32 [00:00<00:00, 256.61it/s]

{'pr_auc': 0.7164389859666139, 'roc_auc': 0.6767631286457192, 'f1': 0.6512488436632747, 'loss': 0.6382618751376867}


In [32]:
model_no_pre = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    embedding_dim=256,
)

In [33]:
# STEP 4: define trainer
trainer = Trainer(model=model_no_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=15,
    optimizer_params = {"lr": 1e-4},
    monitor="pr_auc",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(4031, 256, padding_idx=0)
    (procedures): Embedding(1276, 256, padding_idx=0)
    (drugs): Embedding(194, 256, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=256, out_features=256, bias=False)
              (1): Linear(in_features=256, out_features=256, bias=False)
              (2): Linear(in_features=256, out_features=256, bias=False)
            )
            (output_linear): Linear(in_features=256, out_features=256, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=256, out_features=1024, bias=True)
          

Epoch 0 / 15: 100%|██████████| 243/243 [00:03<00:00, 66.17it/s]

--- Train epoch-0, step-243 ---
loss: 1.2175



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 227.09it/s]

--- Eval epoch-0, step-243 ---
pr_auc: 0.6090
roc_auc: 0.5884
f1: 0.6103
loss: 0.7941
New best pr_auc score (0.6090) at epoch-0, step-243



...

--- Train epoch-14, step-3645 ---
loss: 0.5834



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 269.48it/s]

--- Eval epoch-14, step-3645 ---
pr_auc: 0.6611
roc_auc: 0.6421
f1: 0.6327
loss: 0.6925
New best pr_auc score (0.6611) at epoch-14, step-3645
Loaded best model



Evaluation: 100%|██████████| 32/32 [00:00<00:00, 264.14it/s]

{'pr_auc': 0.697333603874223, 'roc_auc': 0.6722280364429949, 'f1': 0.6672694394213382, 'loss': 0.6479178862646222}
